In [15]:
import pandas as pd
import os
os.system('pip install wikipedia')
os.system('pip install geocoder')
os.system('pip install geopy')
os.system('pip install folium')
import wikipedia as wp
from geopy.geocoders import Nominatim
import folium

In [16]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.columns = ['Postalcode', 'Borough', 'Neighborhood']
df = df.iloc[1:]

In [17]:
foo = lambda a: " , ".join(a) 
df = df.groupby('Postalcode').agg({'Borough': 'first', 'Neighborhood': foo}).reset_index()
   
    
df = df.drop(df[(df.Borough == 'Not assigned')].index)
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        #print(row['Borough'])
        row['Neighborhood'] = row['Borough']

In [18]:
print('The shape of the Dataframe ', df.shape)

The shape of the Dataframe  (103, 3)


In [20]:
print('The shape of the final Dataframe ', df_merge.shape)

The shape of the final Dataframe  (103, 5)


In [21]:
df_merge.groupby('Borough').count()

,Postalcode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [22]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [23]:
from IPython.display import display
Downtown_Toronto = df_merge[df_merge['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.head()
Latitude = 43.654027
Longitude = -79.3802003
map_Downtown_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)
for lat, lng, label in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)
    
# map_Downtown_Toronto.save('map_Downtown_Toronto.html')
# f = codecs.open("map_Downtown_Toronto.html", 'r', 'utf-8')
map_Downtown_Toronto

In [24]:
CLIENT_ID = 'BLLCQDJ141BQCBPOYYMFN5VUIFH5ZR4NDTDFRP52QHQM2CPK' # your Foursquare ID
CLIENT_SECRET = 'G20BXB1EQAM0ZSJF35AOMV12LDF11ODIOH0XRVIATC0OLUZN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

In [25]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
        return(nearby_venues)

In [26]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )
Downtown_Toronto_venues

Rosedale


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [27]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(4, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [28]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Rosedale,4,4,4,4,4,4


In [29]:

print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 3 uniques categories.
